# Minimizing Expectation Value of a VQC

This is a PennyLane beginner level optimization challenge. The objective of the challenge is to minimize the output of a variational quantum circuit (VQC); the VQC output is in fact the result of a fixed measurement and is controlled by a set of trainable parameters.

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
from sympy import Matrix

In [2]:
WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

initial_params = np.random.random(NUM_PARAMETERS)

## Circuit

The VQC of the challenge is defined by the function `variational_circuit(params, hamiltonian)`:
- `params` represents the set of trainable parameters, and
- `hamiltonian` represents the unknown observable on which measurements would be done by the VQC.

In [3]:
def variational_circuit(params, hamiltonian):
    """
    This is a template variational quantum circuit containing a fixed
    layout of gates with variable parameters. To be used as a QNode,
    it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function.

    The output of this circuit is the expectation value of a
    Hamiltonian, somehow encoded in the hamiltonian argument

    Args:
        - params (np.ndarray): An array of optimizable parameters of
          shape (30,)
        - hamiltonian (np.ndarray): An array of real parameters
          encoding the Hamiltonian whose expectation value is
          returned.
    
    Returns:
        (float): The expectation value of the Hamiltonian
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

The circuit has 
- 2 qubits (`WIRES = 2`),
- 5 layers (`LAYERS = 5`), and
- 30 parameters (`NUM_PARAMETERS = LAYERS * WIRES * 3`)
Each layer has 3 trainable parameters per qubit, corresponding to the three rotation angles for a general rotation. This would lead to a total of 30 trainable parameters.

The 30 parameters in `params` are rearranged in a 3-D array of dimensions (`LAYERS`$\times$`WIRES`$\times$`3`). PennyLane's `StronglyEntanglingLayers` applies single-qubit rotations followed by entangling operations in each layer :

$$
U_{\text{layer}_i} = \text{CNOT}_{0, 1} \cdot (U_0 \otimes U_1) \quad \text{where} \quad
U_0 = R(\theta_{i,0}, \phi_{i,0}, \lambda_{i,0}),\;
U_1 = R(\theta_{i,1}, \phi_{i,1}, \lambda_{i,1}).
$$
For the convenience of the reader, the general quantum mechanical rotation operator could be formulated as follows :
$$
R(\theta_i, \phi_i, \lambda_i) = 
R_z(\lambda_i) R_y(\theta_i) R_z(\phi_i) = 
\exp\left(-i \frac{\lambda_i}{2} Z\right) 
\exp\left(-i \frac{\theta}{2} Y\right)
\exp\left(-i \frac{\phi_i}{2} Z\right) 
$$

<div style="border-style:solid; border-width:2px; border-color:#ff2299; padding:0.5em;">

Basically, the role of this parameterized part of the VQC is to prepare the 2-qubit states on which the expectation value of the `hamiltonian` observable would be measured. The observable `hamiltonian` would be fed as a Hermitian matrix (`ndarray`) to the function.

</div>

## Trainer (optimizer)

The `optimize_circuit` function tries minimizing VQC output by finding the gradient direction of the VQC output as a cost function of the 30 parameters, taking a step in that direction and checking if the cost has been reduced or not. Therefor, at each step, it would rerun the VQC and takes its output to compare it with the output from the previous step.
- initiate an instance of the VQC : `dev = qml.device('default.qubit', wires = WIRES)`, `circuit = qml.QNode(variational_circuit, dev)`
- initiate a GradiantDescent optimizer : `opt = qml.GradientDescentOptimizer(stepsize = 0.3)`
- at each iteration :
  - take a step and store the previous cost : `params, prev_cost = opt.step_and_cost(circuit, params, hamiltonian)`
  - compare the previous cost with the new cost : `cost = circuit(params, hamiltonian)`

In [4]:
def optimize_circuit(params,hamiltonian):
    """
    Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit
    function into an executable QNode.
    Next, you should minimize the variational circuit using
    gradient-based optimization to update the input params.
    Return the optimized value of the QNode as a single floating-point
    number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of
          dimension 30
        - hamiltonian (np.ndarray): An array of real parameters
          encoding the Hamiltonian whose expectation value you should
          minimize.
    Returns:
        float: the value of the optimized QNode
    """
    dev = qml.device('default.qubit', wires = WIRES)
    circuit = qml.QNode(variational_circuit, dev)
    # Write your code to minimize the circuit
    opt = qml.GradientDescentOptimizer(stepsize = 0.3)
    max_iterations = 100
    conv_tol = 1e-07

    for n in range(max_iterations):
        params, prev_cost = opt.step_and_cost(circuit, params, hamiltonian)
        params = params[0]
        cost = circuit(params, hamiltonian)
        if np.abs(cost - prev_cost) <= conv_tol:
            break
    
    return circuit(params, hamiltonian)

<div style="border-style:solid; border-width:2px; border-color:#ff2299; padding:0.5em;">

It should be noted that there are optimized versions of gradient descent that address various challenges such as slow convergence, oscillations, and getting stuck in local minima. An overview of these variants could be found in [An overview of gradient descent optimization algorithms](https://arxiv.org/abs/1609.04747).

- [Adagrad (Adaptive Gradient Algorithm)](https://optimization.cbe.cornell.edu/index.php?title=AdaGrad)
- [Nesterov Accelerated Gradient (NAG)](https://mitliagkas.github.io/ift6085-2018/ift-6085-lecture-6-notes.pdf)
- [RMSprop](https://optimization.cbe.cornell.edu/index.php?title=RMSProp)
- [Adam (Adaptive Moment Estimation)](https://optimization.cbe.cornell.edu/index.php?title=Adam)

</div>

## Test run

Now lets test the optimizer with the following $4\times4$ Hermitian matrices as our observables:

In [6]:
display(Matrix([[0.863327072347624, 0.0167108057202516, 0.07991447085492759, 0.0854049026262154], 
         [0.0167108057202516, 0.8237963773906136, -0.07695947154193797, 0.03131548733285282], 
         [0.07991447085492759, -0.07695947154193795, 0.8355417021014687, -0.11345916130631205], 
         [0.08540490262621539, 0.03131548733285283, -0.11345916130631205, 0.758156886827099]]))
print("expected_output: 0.61745341")

display(Matrix([[0.32158897156285354, -0.20689268438270836, 0.12366748295758379, -0.11737425017261123],
                [-0.20689268438270836, 0.7747346055276305, -0.05159966365446514, 0.08215539696259792],
                [0.12366748295758379, -0.05159966365446514, 0.5769050487087416, 0.3853362904758938],
                [-0.11737425017261123, 0.08215539696259792, 0.3853362904758938, 0.3986256655167206]]))
print("expected_output: 0.00246488")

Matrix([
[ 0.863327072347624, 0.0167108057202516, 0.0799144708549276, 0.0854049026262154],
[0.0167108057202516,  0.823796377390614, -0.076959471541938, 0.0313154873328528],
[0.0799144708549276, -0.076959471541938,  0.835541702101469, -0.113459161306312],
[0.0854049026262154, 0.0313154873328528, -0.113459161306312,  0.758156886827099]])

expected_output: 0.61745341


Matrix([
[ 0.321588971562854,  -0.206892684382708,   0.123667482957584, -0.117374250172611],
[-0.206892684382708,   0.774734605527631, -0.0515996636544651, 0.0821553969625979],
[ 0.123667482957584, -0.0515996636544651,   0.576905048708742,  0.385336290475894],
[-0.117374250172611,  0.0821553969625979,   0.385336290475894,  0.398625665516721]])

expected_output: 0.00246488


In [5]:
def run(test_case_input: str) -> str:       
    ins = np.array(json.loads(test_case_input), requires_grad = False)
    hamiltonian = np.array(ins,float).reshape((2 ** WIRES), (2 ** WIRES))
    np.random.seed(1967)
    initial_params = np.random.random(NUM_PARAMETERS)
    out = str(optimize_circuit(initial_params, hamiltonian))    
    return out


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.isclose(solution_output, expected_output, rtol=5e-2)


test_cases = [
    ('[0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]',
    '0.61745341'),
    ('[0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]',
    '0.00246488')
]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")
    try:
        output = run(input_)
    except Exception as exc:
        print(f"Runtime Error. {exc}")
    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")
        else:
            print("Correct!")

Running test case 0 with input '[0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]'...
Correct!
Running test case 1 with input '[0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]'...
Correct!
